# Load dataset

In [ ]:
import json
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving dialogues_016.json to dialogues_016.json
Saving dialogues_017.json to dialogues_017.json
Saving dialogues_019.json to dialogues_019.json
Saving dialogues_018.json to dialogues_018.json


In [ ]:
with open("/content/dialogues_018.json", 'r') as file:
  data_018 = json.load(file)

#data

In [ ]:
with open("/content/dialogues_019.json", 'r') as file:
  data_019 = json.load(file)

In [ ]:
!cat dialogues_018.json dialogues_019.json > dialogues.json

In [ ]:
json.dumps([data_019, data_018])

'[[{"dialogue_id": "19_00000", "services": ["Flights_1"], "turns": [{"frames": [{"actions": [{"act": "INFORM_INTENT", "canonical_values": ["SearchRoundtripFlights"], "slot": "intent", "values": ["SearchRoundtripFlights"]}], "service": "Flights_1", "slots": [], "state": {"active_intent": "SearchRoundtripFlights", "requested_slots": [], "slot_values": {}}}], "speaker": "USER", "utterance": "Can you search for flights for me?"}, {"frames": [{"actions": [{"act": "REQUEST", "canonical_values": [], "slot": "destination_city", "values": []}, {"act": "REQUEST", "canonical_values": [], "slot": "return_date", "values": []}, {"act": "REQUEST", "canonical_values": [], "slot": "origin_city", "values": []}], "service": "Flights_1", "slots": []}], "speaker": "SYSTEM", "utterance": "Where are you leaving from, going, and when are you going to come back?"}, {"frames": [{"actions": [{"act": "INFORM", "canonical_values": ["New York"], "slot": "origin_city", "values": ["NY"]}, {"act": "INFORM", "canonical

Tentando concatenar todos os arquivos em só

In [ ]:
data = json.load(json.dumps([data_019, data_018]))

AttributeError: ignored

Poder ser que seja necessário primeiramente processar o arquivo json antes de converter para um formato aceito pelos modelos de aprendizagem

In [ ]:
df = pd.read_json("/content/dialogues_018.json")
#print(df)

# Data preprocessing

In [21]:
data[0]['turns'][0]

{'frames': [{'actions': [{'act': 'INFORM',
     'canonical_values': ['Premium Economy'],
     'slot': 'seating_class',
     'values': ['Premium Economy']},
    {'act': 'INFORM_INTENT',
     'canonical_values': ['SearchOnewayFlight'],
     'slot': 'intent',
     'values': ['SearchOnewayFlight']}],
   'service': 'Flights_1',
   'slots': [],
   'state': {'active_intent': 'SearchOnewayFlight',
    'requested_slots': [],
    'slot_values': {'seating_class': ['Premium Economy']}}}],
 'speaker': 'USER',
 'utterance': 'I need a one way flight and prefer traveling in Premium Economy class.'}

In [22]:
corpus = [data[0]['turns'][0]['utterance']]
corpus

['I need a one way flight and prefer traveling in Premium Economy class.']



*   Tokenize
*   Create sequence
*   Create input and output



## Feature extraction

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features=2**3, stop_words='english')
X = vectorizer.fit_transform(corpus)
print(X.shape)

(1, 8)


In [24]:
vectorizer

HashingVectorizer(n_features=8, stop_words='english')

ou

## Vectorize

In [26]:
import tensorflow as tf

import numpy as np
import os
import time

In [32]:
chars = tf.strings.unicode_split(corpus, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'I', b' ', b'n', b'e', b'e', b'd', b' ', b'a', b' ', b'o', b'n', b'e',
  b' ', b'w', b'a', b'y', b' ', b'f', b'l', b'i', b'g', b'h', b't', b' ',
  b'a', b'n', b'd', b' ', b'p', b'r', b'e', b'f', b'e', b'r', b' ', b't',
  b'r', b'a', b'v', b'e', b'l', b'i', b'n', b'g', b' ', b'i', b'n', b' ',
  b'P', b'r', b'e', b'm', b'i', b'u', b'm', b' ', b'E', b'c', b'o', b'n',
  b'o', b'm', b'y', b' ', b'c', b'l', b'a', b's', b's', b'.']]>

In [33]:
vocab = list(set(corpus))

In [34]:
display(vocab)

['I need a one way flight and prefer traveling in Premium Economy class.']

In [43]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [46]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]>

In [47]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [48]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]',
  b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]']]>

# Training

https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU <br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

# Testing

# Evaluation